In [1]:
%load_ext autoreload
%autoreload 2

In [87]:
import os
import numpy as np
import json
from dataclasses import dataclass

import torch
import torch.nn.functional as F
from torchvision.utils import save_image
import networkx as nx
from matplotlib import pyplot as plt
import matplotlib.patches as patches
from PIL import Image, ImageDraw
import webcolors

from minimal.imaging import draw_plan
from minimal.layout import LayoutGraphBuilder, LayoutGraph, NodeType
from minimal.gen import generate_plan

%matplotlib inline